In [ ]:
pip install python-docx

In [ ]:
pip install nltk

In [2]:
import pandas as pd
import re
import pathlib
import numpy as np
import pickle
from IPython.display import display
from docx import Document
from pprint import pprint
from collections import defaultdict
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
def get_data():
  df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl')
  df_2 = df.astype(str, errors='ignore')
  df_2 = df_2.applymap(lambda x: x[1:] if x.startswith(".") else x)
  df_2 = df_2.applymap(lambda x: x.strip())
  filt = df_2.Ch.apply(len) < 5
  df_2 = df_2[~filt]
  return df, df_2

In [ ]:
data, data_2 = get_data()

In [ ]:
x = data[data['Lang_Ch'].str.contains("泰雅")]
x['Lang_Ch'].unique()

In [ ]:
data_2.shape

In [ ]:
r = data_2.copy()

def read_a_csv_and_drop_na(path):
    df = pd.read_csv(path, encoding="big5").dropna(how='all').dropna(axis=1, how='all')
    df['Lang_En'] = 'Atayal'
    df['Lang_Ch'] = '泰雅_賽考利克'
    df['From'] = "傳說故事精選篇"
    df = df.rename(columns={"Atayal": "Ab", "Mandarin": "Ch"})
    
    return df

for x in pathlib.Path("../atayal_data/").glob("*.csv"):
    c = read_a_csv_and_drop_na(x)
    r = pd.concat([r, c], ignore_index=True)
    


In [ ]:
r.columns
r.drop('Unnamed: 2', inplace=True,axis=1)

In [ ]:
r.to_pickle('./Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")

In [ ]:
df = pd.read_pickle('./Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")
df

# 02-16 更新:只要泰雅語就好, 將 Rau(1992) 資料加進來

In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")

df_atayal = df[df['Lang_En'] == 'Atayal']
print(df_atayal.shape)

r = df_atayal.copy()

In [ ]:
rau_1992 = pd.read_excel("data/Rau_1992_text_story.xlsx", sheet_name=None)

In [ ]:
rau_1992

In [ ]:
for sheet in rau_1992:
    rau_1992[sheet] = rau_1992[sheet].dropna()
    print("shape of sheet", sheet, rau_1992[sheet].shape)
    rau_1992[sheet] = rau_1992[sheet].rename(columns={"Atayal": "Ab", "English": "Ch"})
    rau_1992[sheet]['Lang_En'] = 'Atayal'
    rau_1992[sheet]['Lang_Ch'] = '泰雅_賽考利克'
    rau_1992[sheet]['From'] = 'Rau (1992)'
    
    r = pd.concat([r, rau_1992[sheet]], ignore_index=True)
    print(r.shape)
    print('======')

In [ ]:
r.shape

In [ ]:
r.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220216.pkl', compression="gzip")

# 02-17 更新: 整理大崁崁群的word檔

In [ ]:
def 檢查各檔案的每個句子的tag(path):

    doc = Document(path)

    having_five_tags = False
    after_title_part = False
    after_new_line = False

    all_sentences = []

    for para in doc.paragraphs:
        
#         print(f'{para.text}\n###')
        
        ################################
        # BEGIN: 處理語料前面的metadata #       
        ################################
        
        if not after_title_part:
            if not para.text.startswith("A:") and\
            not para.text.startswith("RA:") and\
            not para.text.startswith("G:") and\
            not para.text.startswith("M:") and\
            not para.text.startswith("RM:"):
                continue

        after_title_part = True
        
        

        # 遇到換行
        if len(para.text.strip()) == 0:
            after_new_line = True
            all_sentences.append(dict())

        if not para.text.startswith("A:") and\
        not para.text.startswith("RA:") and\
        not para.text.startswith("G:") and\
        not para.text.startswith("M:") and\
        not para.text.startswith("RM:"):
            continue

        for p in para.text.split('\n'):
            if p.startswith("A:") or p.startswith("RA:") or p.startswith("G:") or p.startswith("M:") or p.startswith("RM:"):
                tag = p.split(":")[0]
                content = p.lstrip(tag + ":").strip()
                if content == 'none':
                    content = None

                if len(all_sentences) == 0:
                    all_sentences.append(dict())

                all_sentences[-1][tag] = content

    return all_sentences

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None}])

for x in pathlib.Path("data/泰雅族大嵙崁群的部落故事").glob("*aligned.docx"):
    print(f"[[[{x}]]]")
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)

    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        if 'RA' not in row:
            ab = row['A']
        else:
            if row['RA'] is None:
                ab = row['A']
            else:
                ab = row['RA']

        if 'RM' not in row:
            ch = row['M']
        else:
            if row['RM'] is None:
                ch = row['M']
            else:  
                ch = row['RM']
        
        if ab is None or ch is None:
            continue
            
        results = results.append({'Ab':ab, 'Ch':ch, 'Source': x.name[:4] + "_" + str(index)}, ignore_index = True)

In [ ]:
from IPython.display import display
pd.set_option('display.max_rows', 1000)

results = results.dropna()
results['From'] = '大嵙崁群故事'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

display(results.dropna())

In [ ]:
results[results['Source'] == 'TM01_3']

In [ ]:
import pickle
import pandas as pd

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220216.pkl', compression="gzip")

# df_atayal = df[df['Lang_En'] == 'Atayal']
# print(df_atayal.shape)

r = df.append(results, ignore_index=True)
r

In [ ]:
r.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220217.pkl', compression="gzip")

# Huang 1994

In [ ]:
from docx import Document
from pprint import pprint

import pickle
import pandas as pd



In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220217.pkl', compression="gzip")
doc = Document("data/Huang_1994_text_aligned.docx")
    
results = []

for para in doc.paragraphs:
    print(para.text)
    
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("e:"):
        text = para.text.lstrip("e:").strip()
        results[-1]["Ch"] = text

results = pd.DataFrame(results)
results['From'] = 'Huang 1994'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = df.append(results, ignore_index=True)
df
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl', compression="gzip")

# 加進完整的大嵙崁

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/泰雅族大嵙崁群的部落故事").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
#     display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None else row['RA']
            ch = row['M'] if row['RM'] is None else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        df= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, df], ignore_index = True)

In [ ]:
from IPython.display import display
pd.set_option('display.max_rows', 1000)

results = results.dropna()
results['From'] = '大嵙崁群故事'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

display(results.dropna())


In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl', compression="gzip")
df = pd.concat([df, results], ignore_index=True)
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221102.pkl', compression="gzip")

# 加進復興鄉故事

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/復興鄉故事").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
    all_sentences = all_sentences.replace({np.nan: None})
    display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None or len(row['RA']) == 0 else row['RA']
            ch = row['M'] if row['RM'] is None or len(row['RM']) == 0 else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        df= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, df], ignore_index = True)

In [ ]:
results['From'] = '復興鄉故事'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221102.pkl', compression="gzip")
df = pd.concat([df, results], ignore_index=True)
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221115.pkl', compression="gzip")

In [ ]:
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221117.pkl', compression="gzip")

In [ ]:
list(df[df['Ab'].str.contains("none", na=False)]['Source'])

In [ ]:
for i, row in df.iterrows():

    if isinstance(row['Ab'], str) and 'G:' in row['Ab']:
        print(row['Ab'])
        print('@')

        if row['Ab'].startswith('none'):
            new = row['Ab'].replace('none\nG: ', '').strip()
        else:
            new = re.sub('G:.*','', row['Ab']).strip()
        df.at[i,'Ab'] = new
        print(new)
        print('----')


# 重新整理復興鄉

In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221117.pkl', compression="gzip")

In [ ]:
df.shape

In [ ]:
df = df[~df['From'].str.contains('復興鄉')]

In [ ]:
df.shape

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/復興鄉故事1").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
    all_sentences = all_sentences.replace({np.nan: None})
#     display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None or len(row['RA']) == 0 else row['RA']
            ch = row['M'] if row['RM'] is None or len(row['RM']) == 0 else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        dff= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, dff], ignore_index = True)

results['From'] = '復興鄉故事一'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

In [ ]:
df = pd.concat([df, results], ignore_index=True)

In [ ]:
df.shape

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/復興鄉故事2").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
    all_sentences = all_sentences.replace({np.nan: None})
#     display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None or len(row['RA']) == 0 else row['RA']
            ch = row['M'] if row['RM'] is None or len(row['RM']) == 0 else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        dff= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, dff], ignore_index = True)
results = results.dropna()
        
display(results)
results['From'] = '復興鄉故事二'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

In [ ]:
df = pd.concat([df, results], ignore_index=True)

In [ ]:
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221226.pkl', compression="gzip")

# Egerod 1974 & Huang 1993 & Rau et al. 1995

In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221207.pkl', compression="gzip")

    
results = []

doc = Document("data/Egerod_1974_text.docx")

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("e:"):
        text = para.text.lstrip("e:").strip()
        results[-1]["Ch"] = text   
        
results = pd.DataFrame(results)
results['From'] = 'Egerod (1974)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

In [ ]:
results = []

doc = Document("data/Huang_1993_text.docx")

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("e:"):
        text = para.text.lstrip("e:").strip()
        results[-1]["Ch"] = text   
        
results = pd.DataFrame(results)
results['From'] = 'Huang (1993)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

In [ ]:
results = []

doc = Document("data/Rau_1995_text.docx")

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("m:"):
        text = para.text.lstrip("m:").strip()
        results[-1]["Ch"] = text   
        
results = pd.DataFrame(results)
results['From'] = 'Rau et al. (1995)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

In [ ]:
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

# Huang and Wu 2016

In [4]:
results = []

doc = Document("data/Huang&Wu_text_1_aligned.docx")
i = 0

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip().replace('\xa0', ' ')
        results.append({"Ab": text, "Source": f"1_{i}"})
        i += 1
    elif para.text.startswith("m:"):
        text = para.text.lstrip("m:").strip()
        results[-1]["Ch"] = text   
        
doc = Document("data/Huang&Wu_text_2_aligned.docx")
i = 0

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip().replace('\xa0', ' ')
        results.append({"Ab": text, "Source": f"2_{i}"})
        i += 1
    elif para.text.startswith("m:"):
        text = para.text.lstrip("m:").strip()
        results[-1]["Ch"] = text  

results = pd.DataFrame(results)
results['From'] = 'Huang & Wu (2016)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

results
# df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

,Ab,Source,Ch,From,Lang_En,Lang_Ch
0,"ita’ Tayal hiya’ ga, ini’ balay hmut mqyanux r...",1_0,我們泰雅族生活不隨便，而且習俗也很多。,Huang & Wu (2016),Atayal,泰雅_賽考利克
1,"ta’ qu matas qani hiya’ ga, nyux yan ke’ pin’a...",1_1,關於紋面的習俗，就如同我們祖先的遺訓說：「紋面象徵著能獨立生活的泰雅男孩及泰雅女孩之學習成果。」,Huang & Wu (2016),Atayal,泰雅_賽考利克
2,nanu’ yasa ini’ nha’ hmci balay qu matas qani,1_2,所以紋面是真的不能隨便的。,Huang & Wu (2016),Atayal,泰雅_賽考利克
3,cingay balay qu pinqzywan nha’ squ ’ringan na ...,1_3,關於紋面的起源說法很多。,Huang & Wu (2016),Atayal,泰雅_賽考利克
4,nyux qutux qu pinqzywan nha’ qani.,1_4,這有一則說法是這麼說的。,Huang & Wu (2016),Atayal,泰雅_賽考利克
5,nanu’ ta’ qu ’laqi’ mlikuy ru ’laqi’ kneril na...,1_5,聽說泰雅族的男孩及女孩，到了一定的年齡就必須「會工作」。,Huang & Wu (2016),Atayal,泰雅_賽考利克
6,"son mha “baq mtzywaw” qani hiya mga, ’laqi’ ml...",1_6,對男孩來說，所謂「會工作」就是必須會狩獵、會開墾，有些甚至跟隨出草，然後才會被紋面。,Huang & Wu (2016),Atayal,泰雅_賽考利克
7,"’laqi’ kneril hiya’ ga, siki baq tminun, baq s...",1_7,對女孩來說，所謂「會工作」就是必須會織布、會做家事，然後也才會被紋面。,Huang & Wu (2016),Atayal,泰雅_賽考利克
8,ta’ qu ’laqi’ mlikuy’ ka pintasan rqyas nya’ q...,1_8,紋面的男孩被稱為「真正的男人」，才可以娶妻。,Huang & Wu (2016),Atayal,泰雅_賽考利克
9,’laqi’ kneril ka pintasan rqyas nya’ qasa uzi ...,1_9,紋面的女孩被稱為「真正的女人」，才有人來提親。,Huang & Wu (2016),Atayal,泰雅_賽考利克


In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221226.pkl', compression="gzip")


In [ ]:
a = 檢查各檔案的每個句子的tag('E:/clone/Formosan-languages/data/復興鄉故事2/F210_aligned.docx')
a

In [ ]:
df = df[df['From'] != '復興鄉故事二']

In [ ]:
df.From.unique()

In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221226.pkl', compression="gzip")
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221226-2.pkl', compression="gzip")

In [ ]:
x=df[df['From']=='復興鄉故事二']
zz = x[x['Ab'].str.contains("mkayal ga")]['Ab']
pd.options.display.max_colwidth = 1000
zz.to_string()
display(zz)
# type(zz)

# 重新整理傳說故事

In [112]:
yy = pd.read_excel("data/Y&Y.xlsx", sheet_name=None)

results = []

for story in yy:
    story_id = story.strip()
    for index, rows in yy[story].iterrows():
        if isinstance(rows['Atayal'], float) or len(rows['Atayal'].strip()) == 0:
            continue
        atayal = rows['Atayal'].replace(' \t', ' ').replace('\t', '').strip()
        ch = rows['Mandarin'].strip()
#         if isinstance(ch, float):
#             print(ch)
#             print(index)
#             print(rows)
#             print(story_id)
        results.append({"Ab": atayal, "Source": f"{story_id}_{index}", "Ch": ch})

results = pd.DataFrame(results)
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'
results['From'] = "傳說故事精選篇"

# df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221226-2.pkl', compression="gzip")



In [113]:
df = df[df['From'] != '傳說故事精選篇']
df = pd.concat([df, results], ignore_index=True)

In [ ]:
rau1992 = pd.read_excel("data/Rau_1992_text_story.xlsx", sheet_name=None)

results = []

for story in rau1992:
    story_id = story.lstrip('0')
    for index, rows in rau1992[story].iterrows():
        if isinstance(rows['Atayal'], float) or len(rows['Atayal'].strip()) == 0:
            continue
        atayal = rows['Atayal'].replace(' \t', ' ').replace('\t', '').strip()
        ch = rows['English'].strip()
        results.append({"Ab": atayal, "Ch": ch, "Source": f"{story_id}_{index}"})

results = pd.DataFrame(results)
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'
results['From'] = "Rau (1992)"
results


In [ ]:
df = df[df['From'] != "Rau (1992)"]
df = pd.concat([df, results], ignore_index=True)

In [ ]:
df

In [ ]:
results = []

doc_id = 1
for f in pathlib.Path("data/Rau_et_al_1995").glob("Rau_et_al_1995_text_aligned_0*.docx"):
    print(f)
    doc = Document(f)
    phrase_id = 0
    for para in doc.paragraphs:
        if para.text.startswith("a:"):
            text = para.text.lstrip("a:").strip()
            results.append({"Ab": text, "Source": f"{doc_id}_{phrase_id}"})
            phrase_id += 1
        elif para.text.startswith("m:"):
            text = para.text.lstrip("m:").strip()
            results[-1]["Ch"] = text
    doc_id += 1

results
results = pd.DataFrame(results)
results['From'] = 'Rau et al. (1995)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

In [ ]:
df = df[df['From'] != 'Rau et al. (1995)']
df = pd.concat([df, results], ignore_index=True)

In [114]:
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221227.pkl', compression="gzip")

In [6]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221227.pkl', compression="gzip")
df = pd.concat([df, results], ignore_index=True)
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221227-2.pkl', compression="gzip")